# KDD CUP 2018
https://biendata.com/competition/kdd_2018/

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

root_path = Path('/media/share/data/kaggle/kdd2018')
os.listdir(root_path)

['beijing_17_18_meo.csv',
 'beijing_17_18_aq.csv',
 'Beijing_AirQuality_Stations_en.xlsx']

In [2]:
meo_df = pd.read_csv(str(root_path / 'beijing_17_18_meo.csv'))
meo_df.sample(5)

,station_id,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed,weather
65410,tongzhou_meo,116.756667,39.847500,2017-07-14 04:00:00,31.2,999.6,64,72.0,1.2,Sunny/clear
60464,pinggu_meo,117.117778,40.169444,2017-12-20 22:00:00,-8.1,1024.4,67,188.0,1.0,Sunny/clear
118833,shijingshan_meo,116.205278,39.942500,2017-08-14 01:00:00,24.9,1001.7,79,21.0,0.5,Sunny/clear
133764,daxing_meo,116.354444,39.718611,2017-04-26 11:00:00,21.0,1014.1,11,350.0,2.5,Sunny/clear
49704,shangdianzi_meo,117.111667,40.658889,2017-09-29 09:00:00,23.0,977.0,27,243.0,3.2,Sunny/clear


In [3]:
aq_df = pd.read_csv(str(root_path / 'beijing_17_18_aq.csv'))
aq_df.sample(5)

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
144145,miyunshuiku_aq,2017-03-31 13:00:00,15.0,36.0,11.0,0.2,73.0,3.0
166688,nongzhanguan_aq,2017-11-04 20:00:00,87.0,115.0,78.0,1.6,2.0,4.0
158679,nansanhuan_aq,2017-12-11 17:00:00,5.0,27.0,35.0,0.3,31.0,2.0
37154,dingling_aq,2017-03-16 03:00:00,115.0,152.0,60.0,1.2,67.0,24.0
304276,zhiwuyuan_aq,2017-04-08 04:00:00,51.0,114.0,33.0,0.7,59.0,9.0


In [4]:
aqs_df = pd.read_excel(str(root_path / 'Beijing_AirQuality_Stations_en.xlsx'))
aqs_df.sample(5)

,Pollutant Species,Unnamed: 1,Unnamed: 2
46,Stations Near Traffic,NaN,NaN
12,tiantan_aq,116.407,39.886
10,Urban Stations,NaN,NaN
24,Suburban Stations,NaN,NaN
22,gucheng_aq,116.184,39.914


## EDA
- meo_df: features  
- aq_df: labels

The main goal is predictng air pollution the next day.  

## 1. go through datasets

In [5]:
meo_sid = pd.DataFrame(meo_df['station_id'].str.split('_').str[0])
meo_df1 = meo_df.copy()
meo_df1.drop(['station_id'], 1, inplace=True)
meo_df1 = pd.concat([meo_sid.reset_index(drop=True), meo_df1.reset_index(drop=True)], 1)

print(meo_df1.sample(3))
print(meo_df1['station_id'].unique())
print('Total City: ', meo_df1['station_id'].nunique())

         station_id   longitude   latitude             utc_time  temperature  \
76969      chaoyang  116.500833  39.952500  2017-11-06 22:00:00          5.2   
108819      beijing  116.469444  39.806111  2017-06-23 16:00:00         18.9   
121378  shijingshan  116.205278  39.942500  2017-11-28 02:00:00          5.1   

        pressure  humidity  wind_direction  wind_speed      weather  
76969     1013.7        94           302.0         0.6          Fog  
108819    1001.1        94            18.0         2.4         Rain  
121378    1017.8        19           317.0         9.8  Sunny/clear  
['shunyi' 'hadian' 'yanqing' 'miyun' 'huairou' 'shangdianzi' 'pinggu'
 'tongzhou' 'chaoyang' 'pingchang' 'zhaitang' 'mentougou' 'beijing'
 'shijingshan' 'fengtai' 'daxing' 'fangshan' 'xiayunling']
Total City:  18


In [6]:
aq_sid = pd.DataFrame(aq_df['stationId'].str.split('_').str[0])
aq_df1 = aq_df.copy()
aq_df1.drop(['stationId'], 1, inplace=True)
aq_df1 = pd.concat([aq_sid.reset_index(drop=True), aq_df1.reset_index(drop=True)], 1)

print(aq_df1.sample(3))
print(aq_df1['stationId'].unique())
print('Total City: ', aq_df1['stationId'].nunique())

       stationId             utc_time  PM2.5  PM10   NO2   CO    O3  SO2
211635   tiantan  2017-11-26 13:00:00   11.0  51.0  55.0  0.6   8.0  2.0
138460     miyun  2017-08-31 05:00:00   73.0  74.0  19.0  0.7  81.0  2.0
192182   qianmen  2017-09-17 04:00:00   10.0   NaN  22.0  0.3  75.0  3.0
['aotizhongxin' 'badaling' 'beibuxinqu' 'daxing' 'dingling' 'donggaocun'
 'dongsi' 'dongsihuan' 'fangshan' 'fengtaihuayuan' 'guanyuan' 'gucheng'
 'huairou' 'liulihe' 'mentougou' 'miyun' 'miyunshuiku' 'nansanhuan'
 'nongzhanguan' 'pingchang' 'pinggu' 'qianmen' 'shunyi' 'tiantan'
 'tongzhou' 'wanliu' 'wanshouxigong' 'xizhimenbei' 'yanqin' 'yizhuang'
 'yongdingmennei' 'yongledian' 'yufa' 'yungang' 'zhiwuyuan']
Total City:  35


## 2. match two main dfs

In [7]:
aq_df2 = aq_df1[aq_df1['stationId'].isin(meo_df1['station_id'].unique())]
len(aq_df2)

79974

In [8]:
meo_df2 = meo_df1[meo_df1['station_id'].isin(aq_df2['stationId'].unique())]
len(meo_df2)

79024

## 3. replace outliners with np.nan for further processing

In [9]:
meo_df2.describe()

,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed
count,79024.000000,79024.000000,79024.000000,79024.000000,79024.000000,78907.000000,78907.000000
mean,116.544153,40.053516,25.758271,1023.225244,620.673618,30121.572243,14.627937
std,0.317611,0.237093,3557.270370,3553.716936,23855.217302,170312.137538,3559.926114
min,116.156389,39.718611,-19.500000,983.900000,4.000000,0.000000,0.000000
25%,116.211667,39.847500,2.700000,1002.100000,28.000000,88.000000,1.000000
50%,116.615278,40.126667,14.000000,1010.400000,49.000000,194.000000,1.600000
75%,116.756667,40.223333,23.500000,1018.800000,74.000000,297.000000,2.500000
max,117.117778,40.377500,999999.000000,999999.000000,999999.000000,999999.000000,999999.000000


In [10]:
meo_df2.replace(999999, np.nan, inplace=True)
meo_df2.replace(999017, np.nan, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
meo_df2.describe()

,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed
count,79024.000000,79024.000000,79023.000000,79023.000000,78979.000000,76542.00000,78906.000000
mean,116.544153,40.053516,13.104066,1010.583662,51.256119,184.56160,1.954827
std,0.317611,0.237093,12.081433,10.467580,26.154736,108.48966,1.400447
min,116.156389,39.718611,-19.500000,983.900000,4.000000,0.00000,0.000000
25%,116.211667,39.847500,2.700000,1002.100000,28.000000,86.00000,1.000000
50%,116.615278,40.126667,14.000000,1010.400000,49.000000,189.00000,1.600000
75%,116.756667,40.223333,23.500000,1018.800000,74.000000,289.00000,2.500000
max,117.117778,40.377500,40.300000,1039.700000,100.000000,360.00000,15.400000


In [12]:
aq_df2.describe()

,PM2.5,PM10,NO2,CO,O3,SO2
count,76638.000000,61973.000000,77036.000000,70489.000000,76464.000000,76848.000000
mean,56.949712,88.315137,41.307648,0.937780,61.007167,8.404877
std,65.007655,91.600643,29.720821,0.955609,57.516781,11.778103
min,2.000000,5.000000,1.000000,0.100000,1.000000,1.000000
25%,15.000000,36.000000,18.000000,0.400000,15.000000,2.000000
50%,37.000000,70.000000,34.000000,0.700000,50.000000,4.000000
75%,74.000000,115.000000,60.000000,1.200000,85.000000,10.000000
max,1000.000000,3000.000000,273.000000,13.300000,504.000000,307.000000


## 4. fill nan with mean of forward and backward values
`auto_fillna` and `avg_fillna` are for automatic replacement nan with the mean of forward and backward values  

In [13]:
def avg_fillna(arr):
    # using forward and backward averaging
    ffill = arr.fillna(method='ffill').fillna(method='bfill')
    bfill = arr.fillna(method='bfill').fillna(method='ffill')
    avgfill = (ffill + bfill) / 2
    
    return avgfill


def auto_fillna(df):
    # autofill nan by columns
    for col in df.columns:
        if np.sum(df[col].isnull()) > 0:
            df[col] = avg_fillna(df[col])
            
    return df

In [14]:
meo_df3 = auto_fillna(meo_df2)
aq_df3 = auto_fillna(aq_df2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
meo_df3.describe()

,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed
count,79024.000000,79024.000000,79024.000000,79024.000000,79024.000000,79024.000000,79024.000000
mean,116.544153,40.053516,13.104197,1010.583575,51.238345,184.212296,1.954738
std,0.317611,0.237093,12.081413,10.467543,26.158030,107.876613,1.399539
min,116.156389,39.718611,-19.500000,983.900000,4.000000,0.000000,0.000000
25%,116.211667,39.847500,2.700000,1002.100000,28.000000,86.000000,1.000000
50%,116.615278,40.126667,14.000000,1010.400000,49.000000,188.000000,1.600000
75%,116.756667,40.223333,23.500000,1018.800000,74.000000,287.000000,2.500000
max,117.117778,40.377500,40.300000,1039.700000,100.000000,360.000000,15.400000


In [16]:
aq_df3.describe()

,PM2.5,PM10,NO2,CO,O3,SO2
count,79974.00000,79974.000000,79974.000000,79974.000000,79974.000000,79974.000000
mean,56.76291,89.134681,41.053999,0.945669,61.323561,8.341892
std,64.45208,89.876096,29.563090,0.919112,57.204657,11.770407
min,2.00000,5.000000,1.000000,0.100000,1.000000,1.000000
25%,15.00000,36.000000,18.000000,0.400000,15.000000,2.000000
50%,38.00000,71.000000,34.000000,0.700000,51.000000,4.000000
75%,73.00000,116.000000,59.000000,1.200000,85.000000,10.000000
max,1000.00000,3000.000000,273.000000,13.300000,504.000000,307.000000


## 5. Using Label encoding to reduce memory of process

In [17]:
from sklearn.preprocessing import LabelEncoder

stage1_areaid = aq_df3['stationId'].unique()

le = LabelEncoder().fit(stage1_areaid)

# labelencoding
meo_df3['sid'] = le.transform(meo_df3['station_id'])
aq_df3['sid'] = le.transform(aq_df3['stationId'])

# drop station id
meo_df3.drop(['station_id'], 1, inplace=True)
aq_df3.drop(['stationId'], 1, inplace=True)

print('meo columns: ', meo_df3.columns)
print('aq columns: ', aq_df3.columns)

meo columns:  Index(['longitude', 'latitude', 'utc_time', 'temperature', 'pressure',
       'humidity', 'wind_direction', 'wind_speed', 'weather', 'sid'],
      dtype='object')
aq columns:  Index(['utc_time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2', 'sid'], dtype='object')


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] ==

In [18]:
meo_df3.head(3)

,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed,weather,sid
0,116.615278,40.126667,2017-01-30 16:00:00,-1.7,1028.7,15.0,215.0,1.6,Sunny/clear,7
1,116.615278,40.126667,2017-01-30 17:00:00,-3.5,1028.4,24.0,16.0,1.0,Haze,7
2,116.615278,40.126667,2017-01-30 18:00:00,-3.7,1028.1,27.0,32.0,1.1,Haze,7


In [19]:
aq_df3.head(3)

,utc_time,PM2.5,PM10,NO2,CO,O3,SO2,sid
26658,2017-01-01 14:00:00,352.0,488.0,125.0,5.8,5.0,8.0,0
26659,2017-01-01 15:00:00,375.0,470.0,123.0,5.9,5.0,9.0,0
26660,2017-01-01 16:00:00,364.0,405.0,127.0,6.2,5.0,8.0,0


## objective observation
'Sunny/clear', 'Haze', 'Snow', 'Fog', 'Rain', 'Dust', 'Sand', 'Rain/Snow with Hail', 'Sleet'

In [20]:
meo_df3['weather'].unique()

array(['Sunny/clear', 'Haze', 'Snow', 'Fog', 'Rain', 'Dust', 'Sand',
       'Rain/Snow with Hail', 'Sleet'], dtype=object)